# **Using WEB API to display the movements of a Crypto-Currency**



>In the first step, I will be using the <a href="https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2022-01-01"> CoinGecko API </a> to create a candlestick graphs for Bitcoin.

>I will use the API to get the price data for the last 60 days with 24 observation per day (1 per hour). I will find the max, min, open, and close price per day meaning we will have 60 candlesticks and use that to generate the candlestick graph.

In [21]:
!pip install -q -U watermark

In [25]:
%reload_ext watermark
%watermark -v -p numpy,pandas,matplotlib

Python implementation: CPython
Python version       : 3.8.16
IPython version      : 7.9.0

numpy     : 1.21.6
pandas    : 1.3.5
matplotlib: 3.2.2



In [24]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 73 kB 1.6 MB/s 


In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import plotly.graph_objects as go
from plotly.offline import plot
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

>I am going to start  by getting the data I require. Using the ***get_coin_market_chart_by_id(id, vs_currency, days)***. **id** is the name of the coin I want, **vs_currency** is the currency I want the price in, and **days** is how many days back from today I want.

In [45]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=60)

In [46]:
type(bitcoin_data )

dict

>The response I get is in the form of a **JSON** which includes the **price**, **market caps**, and **total volumes** along with **timestamps** for each observation. I am focused on the ***prices*** so I will select that data.

In [48]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:10]

[[1666612830567, 19445.525176709027],
 [1666616538684, 19357.662700889403],
 [1666620085303, 19377.108301670345],
 [1666623621242, 19260.457427469257],
 [1666627393487, 19298.522499827588],
 [1666630939646, 19275.5170688192],
 [1666634516757, 19290.976376002993],
 [1666638063582, 19323.535606343114],
 [1666641669906, 19338.21954455155],
 [1666645275536, 19348.282096173825]]

In [49]:
#Turn the data into a Pandas DataFrame.
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

>Now that I have the DataFrame I want to convert the **timestamp** to **datetime** and save it as a column called **Date**. I will map my ***unix_to_datetime*** to each **timestamp** and convert it to a readable datetime.

In [50]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [51]:
#Using this modified dataset I can now group by the Date and find the min, max, open, and close for the candlesticks.
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

# ***Great!***

>Finally, I am going to use plotly to create my Candlestick Chart.



In [52]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()